In [49]:
#https://finance.yahoo.com › quote › ETH-BTC
#ETH-BTC - Ethereum B                                     data set use

Bitcoin Prediction using LSTM and tensorflow

In [46]:
#!pip install numpy pandas tensorflow plotly yfinance


In [47]:
#import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import plotly.graph_objects as go

In [15]:
# Load your dataset
file_path = '/content/BTC-USD (1).csv'  # Update with the path to your CSV file
df = pd.read_csv(file_path)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [38]:
df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [16]:
# Define parameters
n_steps = 60  # Number of timesteps (past days) to use for prediction
n_epochs = 50
n_future_days = 7

In [17]:

# Preprocess data
data = df['Close'].values.reshape(-1, 1)

In [21]:
# Scale the data using Min-Max scaling (normalize between 0 and 1)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

X = []
y = []

for i in range(len(data_scaled) - n_steps - n_future_days):
    X.append(data_scaled[i:i+n_steps])
    y.append(data_scaled[i+n_steps:i+n_steps+n_future_days])

X = np.array(X)
y = np.array(y)

In [22]:
# Split data into training and testing sets
split_ratio = 0.8
split_index = int(split_ratio * len(X))

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [24]:
# Build and train the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
model.add(Dense(n_future_days))
model.compile(optimizer='adam', loss='mse')

In [25]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 50)                10400     
                                                                 
 dense_3 (Dense)             (None, 7)                 357       
                                                                 
Total params: 10757 (42.02 KB)
Trainable params: 10757 (42.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
model.fit(X_train, y_train, epochs=n_epochs, batch_size=32)

Epoch 1/50
44/44 [==============================] - 2s 21ms/step - loss: 0.0768
Epoch 2/50
44/44 [==============================] - 1s 23ms/step - loss: 0.0080
Epoch 3/50
44/44 [==============================] - 1s 30ms/step - loss: 0.0032
Epoch 4/50
44/44 [==============================] - 1s 30ms/step - loss: 0.0025
Epoch 5/50
44/44 [==============================] - 1s 22ms/step - loss: 0.0024
Epoch 6/50
44/44 [==============================] - 1s 20ms/step - loss: 0.0023
Epoch 7/50
44/44 [==============================] - 1s 20ms/step - loss: 0.0023
Epoch 8/50
44/44 [==============================] - 1s 20ms/step - loss: 0.0023
Epoch 9/50
44/44 [==============================] - 1s 21ms/step - loss: 0.0021
Epoch 10/50
44/44 [==============================] - 1s 20ms/step - loss: 0.0022
Epoch 11/50
44/44 [==============================] - 1s 21ms/step - loss: 0.0021
Epoch 12/50
44/44 [==============================] - 1s 24ms/step - loss: 0.0020
Epoch 13/50
44/44 [==================

In [27]:
# Make predictions
predicted_prices = model.predict(X_test)

11/11 [==============================] - 0s 8ms/step


In [29]:
# Inverse transform the scaled data to get real values
predicted_prices = scaler.inverse_transform(predicted_prices)

# Reshape y_test to 2D before inverse transform
y_test_reshaped = y_test.reshape(-1, n_future_days)
y_test_real = scaler.inverse_transform(y_test_reshaped)

In [30]:
# Plot candlestick chart for the next day's prediction
fig = go.Figure(data=[go.Candlestick(
                x=df.index[-len(y_test_real):],
                open=df['Open'].values[-len(y_test_real):],
                high=df['High'].values[-len(y_test_real):],
                low=df['Low'].values[-len(y_test_real):],
                close=predicted_prices[:, 0])
                     ])

fig.update_layout(
    title='Next Day Price Prediction',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=True
)

fig.show()

In [33]:
# Make predictions for the next day and the next 7 days
last_sequence = data_scaled[-n_steps:].reshape(1, -1, 1)
next_day_prediction = model.predict(last_sequence)
next_week_prediction = []

for _ in range(n_future_days):
    next_sequence = np.append(last_sequence[:, 1:], next_day_prediction.reshape(1, -1, 1), axis=1)
    next_day_prediction = model.predict(next_sequence)
    next_week_prediction.append(next_day_prediction)

1/1 [==============================] - 0s 23ms/step


In [34]:
# Inverse transform the scaled predictions to get real values
next_day_prediction = scaler.inverse_transform(next_day_prediction)
next_week_prediction = scaler.inverse_transform(np.array(next_week_prediction).squeeze())


In [35]:
# Get the next day's date
next_day_date = df.index[-1] + pd.DateOffset(days=1)

In [36]:
# Print predictions
print("Next Day Prediction:")
print(f"Date: {next_day_date}")
print(f"Predicted Price: {next_day_prediction[0][0]}")

print("\nNext 7 Days Predictions:")
for i in range(n_future_days):
    date = next_day_date + pd.DateOffset(days=i)
    print(f"Date: {date}")
    print(f"Predicted Price: {next_week_prediction[i]}")
    print()


Next Day Prediction:
Date: 2023-09-13 00:00:00
Predicted Price: 38456.93359375

Next 7 Days Predictions:
Date: 2023-09-13 00:00:00
Predicted Price: [27461.03  27699.6   27755.752 27754.791 27900.969 27997.053 28126.633]

Date: 2023-09-14 00:00:00
Predicted Price: [28853.928 29129.605 29196.162 29077.771 29253.957 29302.621 29403.76 ]

Date: 2023-09-15 00:00:00
Predicted Price: [30418.209 30736.207 30805.508 30561.58  30765.732 30770.879 30833.865]

Date: 2023-09-16 00:00:00
Predicted Price: [32172.266 32539.227 32599.9   32222.709 32450.598 32419.812 32432.062]

Date: 2023-09-17 00:00:00
Predicted Price: [34137.426 34581.38  34596.6   34100.01  34317.91  34302.03  34193.85 ]

Date: 2023-09-18 00:00:00
Predicted Price: [36264.    36657.273 36623.285 36010.133 36405.188 36324.652 36057.28 ]

Date: 2023-09-19 00:00:00
Predicted Price: [38456.934 38660.043 38565.973 37833.074 38595.29  38352.3   37946.855]



In [41]:
# Plot candlestick chart for the next day's prediction
next_day_open = df['Close'].values[-n_steps]  # Previous day's closing price
next_day_high = next_day_prediction[0][0]  # Predicted high price
next_day_low = next_day_prediction[0][0]  # Predicted low price
next_day_close = next_day_prediction[0][0]  # Predicted closing price

fig = go.Figure(data=[go.Candlestick(
    x=[next_day_date],
    open=[next_day_open],
    high=[next_day_high],
    low=[next_day_low],
    close=[next_day_close])
])

fig.update_layout(
    title='Next Day Price Prediction',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=True
)

fig.show()



In [45]:
# ...

# Plot candlestick chart for the next 7 days' predictions
fig2 = go.Figure()

for i in range(n_future_days):
    fig2.add_trace(go.Candlestick(
        x=[next_day_date + pd.DateOffset(days=i)],
        open=[df['Close'].values[-n_steps]] + [next_week_prediction[i][0]],
        high=[df['High'].values[-n_steps]] + [next_week_prediction[i][0]],
        low=[df['Low'].values[-n_steps]] + [next_week_prediction[i][0]],
        close=[next_week_prediction[i][0]] + [next_week_prediction[i][0]]
    ))

fig2.update_layout(
    title='Next 7 Days Price Predictions',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=True
)

fig2.show()

Etherium Prediction

In [50]:
# Load your dataset
file_path = '/content/ETH-USD.csv'  # Update with the path to your CSV file
df = pd.read_csv(file_path)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [56]:
df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [51]:
# Define parameters
n_steps = 60  # Number of timesteps (past days) to use for prediction
n_epochs = 50
n_future_days = 7

In [57]:
# Preprocess data
data = df['Close'].values.reshape(-1, 1)

In [58]:
# Scale the data using Min-Max scaling (normalize between 0 and 1)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

X = []
y = []

for i in range(len(data_scaled) - n_steps - n_future_days):
    X.append(data_scaled[i:i+n_steps])
    y.append(data_scaled[i+n_steps:i+n_steps+n_future_days])

X = np.array(X)
y = np.array(y)

In [59]:
# Split data into training and testing sets
split_ratio = 0.8
split_index = int(split_ratio * len(X))

X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [60]:
# Build and train the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
model.add(Dense(n_future_days))
model.compile(optimizer='adam', loss='mse')

In [61]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 50)                10400     
                                                                 
 dense_5 (Dense)             (None, 7)                 357       
                                                                 
Total params: 10757 (42.02 KB)
Trainable params: 10757 (42.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [62]:
model.fit(X_train, y_train, epochs=n_epochs, batch_size=32)

Epoch 1/50
44/44 [==============================] - 2s 20ms/step - loss: 0.0932
Epoch 2/50
44/44 [==============================] - 1s 21ms/step - loss: 0.0436
Epoch 3/50
44/44 [==============================] - 1s 21ms/step - loss: 0.0151
Epoch 4/50
44/44 [==============================] - 1s 29ms/step - loss: 0.0044
Epoch 5/50
44/44 [==============================] - 1s 31ms/step - loss: 0.0032
Epoch 6/50
44/44 [==============================] - 1s 26ms/step - loss: 0.0029
Epoch 7/50
44/44 [==============================] - 1s 20ms/step - loss: 0.0027
Epoch 8/50
44/44 [==============================] - 1s 20ms/step - loss: 0.0026
Epoch 9/50
44/44 [==============================] - 1s 21ms/step - loss: 0.0025
Epoch 10/50
44/44 [==============================] - 1s 20ms/step - loss: 0.0024
Epoch 11/50
44/44 [==============================] - 1s 21ms/step - loss: 0.0023
Epoch 12/50
44/44 [==============================] - 1s 21ms/step - loss: 0.0023
Epoch 13/50
44/44 [==================

In [63]:
# Make predictions
predicted_prices = model.predict(X_test)

12/12 [==============================] - 0s 8ms/step


In [64]:
# Inverse transform the scaled data to get real values
predicted_prices = scaler.inverse_transform(predicted_prices)

# Reshape y_test to 2D before inverse transform
y_test_reshaped = y_test.reshape(-1, n_future_days)
y_test_real = scaler.inverse_transform(y_test_reshaped)

In [67]:
# Plot candlestick chart for the next day's prediction
fig = go.Figure(data=[go.Candlestick(
                x=df.index[-len(y_test_real):],
                open=df['Open'].values[-len(y_test_real):],
                high=df['High'].values[-len(y_test_real):],
                low=df['Low'].values[-len(y_test_real):],
                close=predicted_prices[:, 0])
                     ])

fig.update_layout(
    title='Next Day Price Prediction',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=True
)

fig.show()

In [68]:
# Make predictions for the next day and the next 7 days
last_sequence = data_scaled[-n_steps:].reshape(1, -1, 1)
next_day_prediction = model.predict(last_sequence)
next_week_prediction = []

for _ in range(n_future_days):
    next_sequence = np.append(last_sequence[:, 1:], next_day_prediction.reshape(1, -1, 1), axis=1)
    next_day_prediction = model.predict(next_sequence)
    next_week_prediction.append(next_day_prediction)

1/1 [==============================] - 0s 80ms/step


In [69]:
# Inverse transform the scaled predictions to get real values
next_day_prediction = scaler.inverse_transform(next_day_prediction)
next_week_prediction = scaler.inverse_transform(np.array(next_week_prediction).squeeze())

In [70]:
# Get the next day's date
next_day_date = df.index[-1] + pd.DateOffset(days=1)

In [71]:
# Print predictions
print("Next Day Prediction:")
print(f"Date: {next_day_date}")
print(f"Predicted Price: {next_day_prediction[0][0]}")

print("\nNext 7 Days Predictions:")
for i in range(n_future_days):
    date = next_day_date + pd.DateOffset(days=i)
    print(f"Date: {date}")
    print(f"Predicted Price: {next_week_prediction[i]}")
    print()

Next Day Prediction:
Date: 2023-09-14 00:00:00
Predicted Price: 2446.832763671875

Next 7 Days Predictions:
Date: 2023-09-14 00:00:00
Predicted Price: [1733.5587 1725.1033 1725.3174 1739.4786 1754.377  1756.4182 1786.8663]

Date: 2023-09-15 00:00:00
Predicted Price: [1824.1312 1825.3351 1819.2559 1838.9503 1854.3044 1847.5132 1883.7302]

Date: 2023-09-16 00:00:00
Predicted Price: [1924.5564 1936.6364 1923.9496 1949.2611 1965.1317 1947.7098 1990.9844]

Date: 2023-09-17 00:00:00
Predicted Price: [2035.6083 2059.0686 2040.1427 2070.1152 2086.916  2057.9438 2107.997 ]

Date: 2023-09-18 00:00:00
Predicted Price: [2158.0166 2193.828  2168.736  2202.8357 2220.7954 2178.6147 2236.05  ]

Date: 2023-09-19 00:00:00
Predicted Price: [2294.7898 2344.8103 2311.5283 2351.305  2370.7986 2314.8623 2379.312 ]

Date: 2023-09-20 00:00:00
Predicted Price: [2446.8328 2512.5042 2467.3792 2516.5195 2538.1    2470.8145 2538.6362]



In [72]:
# Plot candlestick chart for the next day's prediction
next_day_open = df['Close'].values[-n_steps]  # Previous day's closing price
next_day_high = next_day_prediction[0][0]  # Predicted high price
next_day_low = next_day_prediction[0][0]  # Predicted low price
next_day_close = next_day_prediction[0][0]  # Predicted closing price

fig = go.Figure(data=[go.Candlestick(
    x=[next_day_date],
    open=[next_day_open],
    high=[next_day_high],
    low=[next_day_low],
    close=[next_day_close])
])

fig.update_layout(
    title='Next Day Price Prediction',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=True
)

fig.show()


In [74]:
# ...

# Plot candlestick chart for the next 7 days' predictions
fig2 = go.Figure()

for i in range(n_future_days):
    fig2.add_trace(go.Candlestick(
        x=[next_day_date + pd.DateOffset(days=i)],
        open=[df['Close'].values[-n_steps]] + [next_week_prediction[i][0]],
        high=[df['High'].values[-n_steps]] + [next_week_prediction[i][0]],
        low=[df['Low'].values[-n_steps]] + [next_week_prediction[i][0]],
        close=[next_week_prediction[i][0]] + [next_week_prediction[i][0]]
    ))

fig2.update_layout(
    title='Next 7 Days Price Predictions',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=True
)

fig2.show()